In [1]:
import os
from aita.datasource.snowflake import SnowflakeDataSource
from aita.datasource.postgresql import PostgreSqlDataSource
from aita.agent import SqlAgent, PandasAgent, PythonAgent

In [2]:
SNOWFLAKE_USER = os.environ.get("SNOWFLAKE_USER")
SNOWFLAKE_PASSWORD = os.environ.get("SNOWFLAKE_PASSWORD")
SNOWFLAKE_ACCOUNT = os.environ.get("SNOWFLAKE_ACCOUNT")
SNOWFLAKE_WAREHOUSE = os.environ.get("SNOWFLAKE_WAREHOUSE")
SNOWFLAKE_DATABASE = os.environ.get("SNOWFLAKE_DATABASE")
SNOWFLAKE_SCHEMA = os.environ.get("SNOWFLAKE_SCHEMA")
SNOWFLAKE_ROLE = os.environ.get("SNOWFLAKE_ROLE")

In [3]:
sf_datasource = SnowflakeDataSource(
    user=SNOWFLAKE_USER,
    password=SNOWFLAKE_PASSWORD,
    account=SNOWFLAKE_ACCOUNT,
    warehouse=SNOWFLAKE_WAREHOUSE,
    database=SNOWFLAKE_DATABASE,
    schema=SNOWFLAKE_SCHEMA,
    role=SNOWFLAKE_ROLE
)

In [4]:
pg_datasource = PostgreSqlDataSource(
    user="",
    password="",
    host="localhost",
    port="5432",
    database="aita"
)

In [6]:
# Basic example of using the SQL agent
sql_agent = SqlAgent(sf_datasource, "gpt-3.5-turbo", 0.8, allow_extract_metadata=True)

print(sql_agent.chat("I want to get the top customers which making the most purchases"))


ProgrammingError: 002003 (02000): SQL compilation error:
Database 'DEMO' does not exist or not authorized.

In [ ]:
# Example of using the SQL agent to run a tool such as querying the data source
sample_sql_query = """
SELECT c_custkey, c_name, SUM(o_totalprice) AS total_purchase
FROM snowflake_sample_data.tpch_sf1.customer
JOIN snowflake_sample_data.tpch_sf1.orders
ON c_custkey = o_custkey
GROUP BY c_custkey, c_name
ORDER BY total_purchase
DESC LIMIT 10
"""

tool_spec = {
    "name": "sql_datasource_query",
    "args": {
        "query": sample_sql_query
    }
}
print(sql_agent.run_tool(tool_spec))

# or rather specify the allow_run_tool=True in the chat method
print(sql_agent.chat("extract snowflake data source metadata", allow_run_tool=True))

In [ ]:
# Example of using the Pandas agent
df1 = sf_datasource.to_pandas("SELECT * FROM CUSTOMER")

pandas_agent = PandasAgent({"df1": df1}, "gpt-3.5-turbo", 0.8)
print(pandas_agent.chat("find the top 5 customers make the most purchases", allow_run_tool=True))


In [ ]:
# Example of using the Python agent
python_agent = PythonAgent(sf_datasource, "gpt-3.5-turbo", 0.8)
python_agent.chat("python code to show the customers data with snowflake database as data source", allow_run_tool=True)